In [1]:
from IPython.display import HTML, display
from IPython.core.debugger import set_trace, Pdb
from tabulate import tabulate
from ipywidgets import interact, IntSlider, Textarea

runcall = Pdb().runcall
# set breakpoint in code: set_trace()
# call function with arguments: runcall(computeLargestSuffixBorderIndexAndShift, "argument")
# see: http://frid.github.io/blog/2014/06/05/python-ipdb-cheatsheet/

# Boyer-Moore String Search Algorithm

Searches for a `pattern` in `text`. Assumes that `text` is much larger than `pattern`.

Core observations:

 * Match `pattern` right-to-left, from the last character to the first.
 
     ```
     ...textual... <- text
        pattern    <- pattern
        7654321    <- char test order
     ```
 
 * Defines a set of _shift rules_ to maximize the amount of characters that can be skipped when a mismatch is found.
 
 These rules reason about what parts of `text` can or cannot ever match `pattern`. These conditions are conservative to avoid missing matches but they can still speedup the search considerably.

## Shift Rules

The final shift is the `max` of applying the following two rules: [bad character rule](#1.-Bad-Character-Rule) and [good suffix rule](#2.-Good-Suffix-Rule).

Note that the rules are only applied when a mismatch occurs.

### 1. Bad Character Rule

Consider the `mismatched-char` in `text` that failed the comparison process, i.e. the `mismatched-char` does not occur in `pattern` at that specific position. We have the following two cases:

* The `mismatched-char` **does not occur** in `pattern`.

    Then shift `pattern` past the `mismatched-char`. Naturally, the `pattern` cannot occur with a character that is not in `pattern`. We can therefore ignore the part of the `text` to the left and including the `mismatched-char`.

    _Example 1:_
    ```
    ...potato... <- text
       dog       <- 't' does not appear in 'dog'
          dog    <- next position skips checks on 'po' and moves past 't'
    ```

    _Example 2:_
    ```
    ...ralopithecu... <- text
       sandpit        <- 'o' does not appear in 'sandpit'
           sandpit    <- next position skips checks on 'sand' and moves past 'o'
    ```

* The `mismatched-char` **does occur** in `pattern`.

    Then align the `pattern` with the closest occurrence of the `mismatched-char` to the left of the current position in `pattern`. If it were not to the left of the current position, then we could go back to a previous search state and we could not guarantee the search algorithm's progress.
    It is sound to skip these positions since `pattern` cannot appear in this position, and the skipped characters are the minimum to align the `mismatched-char`.
   
    _Example 1:_
    ```
    ...longhory... <- text
         horn      <- 'o' does not appear in that position but does occur in 'horn'
           horn    <- next position skips 2 positions
    ```
    
    _Example 2:_
    ```
    ...bajhooes...  <- text
       potatoes     <- 'o' does not appear in that position
           potatoes <- shift to the next 'o' to the left of the current position
    ```

#### Computing The Bad Character Rule

We want to avoid having to search for the last position of the character on each mismatch, so instead we precompute a table to perform that lookup more quickly.

__A. Precompute the full 2D lookup table__

Indexed by the position of the `character` in the alphabet and by the `index` in the `pattern`.

To keep the representation shorter, we will omit all alphabet characters that do not appear in the `pattern`.

In [2]:
def precompute2D(pattern):
    l = list(pattern)
    res = []
    row = {char: -1 for char in l}
    for (index, char) in enumerate(l):
        row[char] = index
        res = res + [dict(row)]
    return res

@interact(pattern='hello')
def showPrecompute2D(pattern):
    res = precompute2D(pattern)
    
    keys = res[0] if res else {}
    table = [ [index] + [ d[k] for k in keys ] for (index, d) in enumerate(res) ]
    
    display(HTML('<h5>last index of char left of position</h5>'))
    return HTML(tabulate(
        [[''] + list(keys.keys())] + table,
        tablefmt='html'
    ))

interactive(children=(Text(value='hello', description='pattern'), Output()), _dom_classes=('widget-interact',)…

__B. Precompute a 1D lookup table with an approximation of the previous table__

Shrinks the 2D table into a 1D array. Only uses the last occurrence of a character. As a result, if a character occurs before its last position (for instance the first `o` in `potato`) the shift in the table will be incorrect. Instead, in that situation, the shift to use should be `1` since that is the largest possible shift that still ensures correctness.

In [3]:
def computeBadCharacterTable(pattern):
    return {char: index for (index, char) in enumerate(list(pattern))}

@interact(pattern='hello')
def show(pattern):
    t = computeBadCharacterTable(pattern)
    
    patternTable = [['pattern:'] + list(pattern),
                    ['index:'] + list(range(len(pattern)))]
    display(HTML(tabulate(patternTable, tablefmt='html')))
    display(HTML('pattern length: ' + str(len(pattern))))
    display(HTML('<h5>last index of letter</h5>'))
    table = [["letter:"] + list(t.keys()),
             ["last-index:"] + list(t.values())]
    return HTML(tabulate(table, tablefmt='html'))

interactive(children=(Text(value='hello', description='pattern'), Output()), _dom_classes=('widget-interact',)…

### 2. Good Suffix Rule

This rule handles the case of when a partial match is found. Thus, `pattern` contains a suffix that is in the `text` we are searching on. The shift distance is computed based on the following two cases:

* The full suffix appears somewhere else in the `pattern`.
* Only part of the suffix appears at the beginning of the `pattern`.

To understand how to compute this shift distance, we will first discuss some relevant background.

#### [Background] Borders

Consider all prefixes and suffixes of a `pattern` such that they are not equal to the `pattern` itself (i.e. they are missing the last/first character in `pattern`, respectively) and call them _proper_ prefix and _proper_ suffix. Then a border is a `string` that is both a proper prefix and a proper suffix of a `pattern`.

(See http://www.inf.fh-flensburg.de/lang/algorithmen/pattern/kmpen.htm#section2 for a detailed explanation on borders.)

In [4]:
def getProperPrefixes(pattern):
    return [pattern[:i] for i in range(0,len(pattern))]

def getProperSuffixes(pattern):
    return [pattern[i:] for i in range(1,len(pattern)+1)]

def getBorders(pattern):
    properPrefixes = getProperPrefixes(pattern)
    properSuffixes = getProperSuffixes(pattern)
    borders = set(properPrefixes).intersection(set(properSuffixes))
    return (properPrefixes, properSuffixes, borders)

@interact(pattern = 'abacab')
def showBorders(pattern):
    # show empty string as empty symbol and wrap string in single quotes for clarity    
    format = lambda stringList: ['ε' if len(s) == 0 else "'{}'".format(s) for s in stringList]
    
    (properPrefixes, properSuffixes, borders) = getBorders(pattern)
    
    return HTML(tabulate(
        [
            ['proper prefixes:'] + format(properPrefixes),
            ['proper suffixes:'] + format(properSuffixes),
            ['borders:'] + format(list(borders))
        ],
        tablefmt='html'))


interactive(children=(Text(value='abacab', description='pattern'), Output()), _dom_classes=('widget-interact',…

#### [Background] Largest Border Widths for each Prefix

It is possible to compute the largest border width for each prefix using the algorithm in http://www.inf.fh-flensburg.de/lang/algorithmen/pattern/kmpen.htm#section2. Although for Boyer-Moore we are interested in suffix matching, the prefix variant of the algorithm is simpler but shares similar principles to the suffix version.

In [5]:
def computeLargestBorderWidths(pattern):
    patternLength = len(pattern)
    borderWidths = [0] * (patternLength + 1)
    # prefix of length 0 has no border, thus width is -1      
    borderWidths[0] = -1
    
    i = 0
    j = -1
    # go over each possible prefix from 0 to patternLength    
    while i < patternLength:
        # 'j' carries the previous border width (initially none, i.e. -1).
        # Iterate over previous borders until we find one that can be extended
        # with the new character at 'pattern[i]'.
        # Note that the index of the next char in the border is the same
        # as its width since we are indexing from 0.
        while j >= 0 and pattern[i] != pattern[j]:
            j = borderWidths[j]
        # if border cannot be extended, then 'j' will be -1
        # thus 'borderWidths[i] = 0' if no border exists
        j += 1
        i += 1
        borderWidths[i] = j
    return borderWidths

Note that:
 * `i` goes over each individual character position in `pattern`
 * `j` is the next character of the previous border (if it exists).
 * The next character of the border will be at the exact position of the border width since the pattern starts at 0. Just like the first index out-of-bounds of the `pattern` is at `patternLength`. Thus, if a border has length 3 (i.e. from indexes 0 to 2 in pattern) then the next character after that border is at index 3.
 * Each loop of the algorithm will try to extend a previous border with the character at `pattern[i]`. And, the border cannot be extended, it will iterate over previous borders until the base case (`borderWidths[0] = -1`) is reached. Each previous border is stored at `borderWidths[j]` since we want previous borders of that prefix.

__Examples using the IPython debugger:__

In [6]:
def printState(vars):
    patternValue = lambda index: "" if index < 0 or index >= (len(vars['pattern'])) else vars['pattern'][index]
    table = tabulate([
            ['pattern:'] + list(vars['pattern']),
            ['index:'] + list(range(len(vars['pattern']))),
            ['borderWidths'] + vars['borderWidths'],
#         prefix from [0 .. i]
            ['i:', vars['i'], patternValue(vars['i'])],
#         border from [0 .. j]
            ['j:', vars['j'], patternValue(vars['j'])]
        ], tablefmt='grid')
    print(table)

# Once you uncomment the examples below, the debugger will be triggered and stop all other cells.
# Type 'exit' to exit the debugger.
    
# Use the following ipdb command to step to next statement and print state once in the debugger.
# ipdb> n;;pp printState(locals())

# Example 1: Successfully extending previous border.
# runcall(computeLargestBorderWidths, "okamoka")
# ipdb> b 10, i == 4;;continue;;pp printState(locals())

# Example 2: Failure to extend previous border, with no existing previous border.
# runcall(computeLargestBorderWidths, "okamokato")
# ipdb> b 10, i == 7;;continue;;pp printState(locals())

# Example 3: Failure to extend previous border, with existing previous border.
# runcall(computeLargestBorderWidths, "aaabaac")
# ipdb> b 10, i == 6;;continue;;pp printState(locals())

__Result Tabled Representation:__

In [7]:
@interact(pattern = 'ababaa')
def showComputeLargestPrefixBorderWidths(pattern):
    borderWidths = computeLargestBorderWidths(pattern)
    
    display(HTML('<h5>Precompute Prefixes Border Widths:</h5>'))
    display(HTML(tabulate(
        [
            ['pattern:'] + list(pattern),
            ['indexes:'] + list(range(0, len(pattern)+1)),
            ['borderWidths:'] + borderWidths
        ],
        tablefmt='html')))
    
    display(HTML('<h5>Meaning with underlined largest prefix border:</h5>'))
    examples = []
    for i in range(0, len(borderWidths)):
        width = borderWidths[i]
        borderEnd = max(width,0)
        border = pattern[0:borderEnd]
        prefixAfterBorderEnd = pattern[borderEnd:i]
        examples.append([i, width, '<u>{}</u>{}'.format(border, prefixAfterBorderEnd)])
    
    display(HTML(tabulate(examples, tablefmt='html', headers=['i', 'width', 'prefix'])))

interactive(children=(Text(value='ababaa', description='pattern'), Output()), _dom_classes=('widget-interact',…

Note the meaning of the border indexes: for a prefix ending at index `i` (i.e. without the current character), the width of the border is `borderWidths[i]`. Thus, when we have pattern `okamoka` with `borderWidths[5] = 1` it means that for prefix `okamo` of the pattern there is a border of length `1` i.e. `o`.

#### 1.2.1. Precomputing the Borders of Suffixes of a Pattern

For Boyer-Moore we need to instead look at suffixes.

In [8]:
def computeLargestSuffixBorderIndexAndShift(pattern):
    patternLength = len(pattern)
    
    shift = [0] * (patternLength + 1)
    
    borderIndex = [0] * (patternLength + 1)
    borderIndex[patternLength] = patternLength + 1

    i = patternLength
    j = patternLength + 1
    # go over each possible suffix from right-to-left     
    while i > 0:
        while j <= patternLength and pattern[i-1] != pattern[j-1]:
            # mismatch occurred, and no previous shift is known, then keep this shift
            if shift[j] == 0:
                shift[j] = j - i
            j = borderIndex[j]
            
        # if border cannot be extended, then 'j' will be (patternLenght + 1)
        # thus 'borderIndex[i] = (patternLenght + 1)' if no border exists
        i -= 1
        j -= 1
        borderIndex[i] = j
    return (borderIndex, shift)

* `borderIndex[i]` contains the starting index in `pattern` of the widest border of the suffix of `pattern` beginning at index `i`. Since these are suffixes, the ending position corresponds to the length of `pattern`.
In other words, for `suffix = pattern[from i to end]` its widest border is `pattern[from borderIndex[i] to end]`.

__Examples with IPython debugger:__

In [13]:
def printState(vars):
    patternValue = lambda index: "" if index < 0 or index >= (len(vars['pattern'])) else vars['pattern'][index]
    table = tabulate([
            ['pattern:'] + [ (str(k) + ':' + v) for (k,v) in zip(list(range(len(vars['pattern']))), list(vars['pattern']))],
            ['borderIndex'] + vars['borderIndex'],
            ['shift'] + vars['shift'],
#         suffix from [i to end]
#         border from [j to end]
            ['i:', vars['i'], patternValue(vars['i'])],
            ['j:', vars['j'], patternValue(vars['j'])]
        ], tablefmt='grid')
    print(table)

# Once you uncomment the examples below, the debugger will be triggered and stop all other cells.
# Type 'exit' to exit the debugger.
    
# Use the following ipdb command to step to next statement and print state once in the debugger.
# ipdb> n;;pp printState(locals())

# TODO: add actual examples
# Example 1: successful border extensions
# Example 2: failed to extend existing border, no other borders to iterate
# Example 3: failed to extend existing border, iterate over several borders
# Example 4: shift that is not stored since there is a shorter shift

# runcall(computeLargestSuffixBorderIndexAndShift, "okamoka")
# ipdb> b 12, i == 2;;continue;;pp printState(locals())


In [10]:
@interact(pattern = 'abbabab')
def showComputeLargestSuffixBorderIndexesAndShift(pattern):
    (borderPosition, shift) = computeLargestSuffixBorderIndexAndShift(pattern)
    
    display(HTML('<h5>Precompute Suffix Border Indexes:</h5>'))
    display(HTML(tabulate(
        [
            ['pattern:'] + list(pattern),
            ['indexes:'] + list(range(0, len(pattern)+1)),
            ['suffix border index:'] + borderPosition,
            ['shift:'] + shift
        ],
        tablefmt='html')))
    
    display(HTML('<h5>Meaning with underlined largest suffix border:</h5>'))
    examples = []
    for i in range(0, len(borderPosition)):
        # start index of border         
        borderStartIndex = borderPosition[i]
        suffix = pattern[borderStartIndex:]
        prefix = pattern[i:borderStartIndex]

        examples.insert(0, [i, borderStartIndex, '{}<u>{}</u>'.format(prefix, suffix), shift[i]])
    
    display(HTML(tabulate(examples, tablefmt='html', headers=['i', 'border<br/>position', 'suffix', 'shift'])))

interactive(children=(Text(value='abbabab', description='pattern'), Output()), _dom_classes=('widget-interact'…

In [11]:
def computeSuffixLargestShift(shift, borderIndex, patternLength):
    # copy input shift into finalShift to avoid mutations
    finalShift = list(shift);

    # widest border of the full pattern
    widestBorderIndex = borderIndex[0];
    
    # go over all suffixes from largest to smallest, i.e. start with suffix[from 0 to end].
    for i in range(0, patternLength+1):
        # when no shift was assigned yet, give it the largest currently known
        if finalShift[i] == 0:
            finalShift[i] = widestBorderIndex
        # when we reach the 'widest border index' we must get the next
        # widest since the previous value would be outside the bounds of the suffix
        # (i.e. would start before the next suffix start index)
        if i == widestBorderIndex:
            widestBorderIndex = borderIndex[widestBorderIndex]

    return finalShift

@interact(pattern = 'abbabab')
def show(pattern):
    m = len(pattern)
    (borderIndex,shift) = computeLargestSuffixBorderIndexAndShift(pattern)
    finalShift = computeSuffixLargestShift(shift, borderIndex, m)
    
    table = [["indexes:"] + list(range(0, m+1)),
             ["pattern:"] + list(pattern),
             ["borderIndex:"] + borderIndex,
             ["shift:"] + shift,
             ["finalShift:"] + finalShift ]
    return HTML(tabulate(table, tablefmt='html'))

interactive(children=(Text(value='abbabab', description='pattern'), Output()), _dom_classes=('widget-interact'…

Where `borderIndex` is the border index array, with values such that `borderIndex[i]` is the left-most border index in the pattern.


Where `finalShift[i]` contains the shift distance of the pattern if a mismatch at position `i – 1` occurs, i.e. if the suffix of the pattern starting at position `i` has matched.

## Final Algorithm

In [12]:
def bmPreprocess(pattern):
    badCharacterTable = computeBadCharacterTable(pattern)
    (borderIndex,shift) = computeLargestSuffixBorderIndexAndShift(pattern)
    goodSuffixTable = computeSuffixLargestShift(shift, borderIndex, len(pattern))
    return (badCharacterTable, goodSuffixTable)

def bmSearch(pattern, text):
    result = []
    patternLength = len(pattern)
    textLength = len(text)
    (badCharacterTable, goodSuffixTable) = bmPreprocess(pattern)
    
    i = 0
    j = 0
    # search from start to end, while pattern fits in text     
    while i <= (textLength-patternLength):
        j = patternLength-1
        while j>=0 and pattern[j] == text[i+j]:
            j -= 1
            
        if j<0:
            result.append(i)
            i += goodSuffixTable[0]
        else:
            # j+1 was the previous char that matched in pattern
            # (i.e. j is the mismatch)
            goodSuffixRule = goodSuffixTable[j+1]
            # the character in the text that failed is at 'i+j'
            # if no entry is found use -1 as default.
            # Note that the badCharacterRule may return a negative number.
            # For instance:
            # pattern: putato
            #    text: potato
            # this would cause the algorithm to not terminate, if there was no goodSuffixRule
            # to cover this situation as the second bad character implies there must exist a
            # suffix. Thus, the goodSuffixRule should have a positive value for shifting.
            badCharacterRule = j-badCharacterTable.get(text[i+j], -1)
            i += max(goodSuffixRule, badCharacterRule)

    return result

@interact(pattern = 'dog', text = Textarea(value='my dog does not like other dogs'))
def showBM(pattern, text):
    patternLength = len(pattern)
    if patternLength <= 0:
        return "ERROR: Pattern must be non-empty."
    indexes = bmSearch(pattern, text)
    
    cutText = text
    result = []
    for i in reversed(indexes):
        matchedPattern = cutText[i:i+patternLength]
        afterPattern = cutText[i+patternLength:]
        result = ['<b><u>', matchedPattern, '</u></b>', afterPattern] + result
        cutText = cutText[0:i]
        
    result = ''.join([cutText] + result)
    display(HTML(result))
    
    return indexes

interactive(children=(Text(value='dog', description='pattern'), Textarea(value='my dog does not like other dog…

## References

1. Article with detailed explanation: http://www.inf.fh-flensburg.de/lang/algorithmen/pattern/bmen.htm
1. Original paper: http://www.cs.utexas.edu/~moore/publications/fstrpos.pdf
1. Wikipedia article: https://en.wikipedia.org/wiki/Boyer–Moore_string_search_algorithm
1. Visualization: http://whocouldthat.be/visualizing-string-matching/
1. https://stackoverflow.com/questions/13175739/what-are-the-shift-rules-for-boyer-moore-string-search-algorithm
1. http://stackoverflow.com/questions/19345263/boyer-moore-good-suffix-heuristics
1. https://www.geeksforgeeks.org/pattern-searching-set-7-boyer-moore-algorithm-bad-character-heuristic/
1. https://www.geeksforgeeks.org/boyer-moore-algorithm-good-suffix-heuristic/
1. https://www.youtube.com/watch?v=NinWEPPrkDQ
1. http://www.cs.uku.fi/~kilpelai/BSA05/lectures/slides03.pdf
1. Article: https://dzone.com/articles/algorithm-week-boyer-moore
1. Implementation with only last char: https://people.ok.ubc.ca/ylucet/DS/BoyerMoore.html